In [426]:
import pennylane as qml
from pennylane import numpy as np

# 양자 장치 설정
dev = qml.device('default.mixed', wires=2)


@qml.qnode(dev)
def random_circuit(params):
    """임의의 게이트를 포함하는 기본 회로"""

    for i in range(10):
        qml.RX(params[i*3+0], wires=0)
        qml.RY(params[i*3+1], wires=1)
        qml.CNOT(wires=[0, 1])
        qml.RZ(params[i*3+2], wires=0)
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))


In [427]:
import pennylane as qml
from pennylane import numpy as np



def extra_polation(circ,theta,noise_factor=[0,1,2,3],p1=0.01,p2=0.02):
    
    real_vaule = circ(theta)
    dev = circ.device
    ops = circ.qtape.operations
    ops_inv = ops[::-1]


    meas = circ.qtape.measurements[0]
    def noise_circ():
        for op in ops:
            eval(f'qml.{op}')
            if len(op.wires)>1:
                for wire in op.wires:
                    qml.DepolarizingChannel(p1, wires=wire)
                    qml.BitFlip(p2, wires=wire)
            else:
                qml.DepolarizingChannel(p1, wires=op.wires) 
                qml.BitFlip(p2, wires=op.wires)
    def noise_circ_inv():
        for op in ops_inv:
            eval(f'qml.adjoint(qml.{op})')
            if len(op.wires)>1:
                for wire in op.wires:
                    qml.DepolarizingChannel(p1, wires=wire)
                    qml.BitFlip(p2, wires=wire)
            else:
                qml.DepolarizingChannel(p1, wires=op.wires) 
                qml.BitFlip(p2, wires=op.wires)
    
    @qml.qnode(dev)
    def extra_polation(factor):
        Z = qml.PauliZ
        Y = qml.PauliY
        X = qml.PauliX
        for i in range(factor):
            noise_circ()
            noise_circ_inv()
        noise_circ()
        return eval(f"qml.{meas}")
    




    res = []
    for factor in noise_factor:
        res.append(extra_polation(factor))
    return res,real_vaule

In [428]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
# 데이터셋 생성 클래스
class QuantumData(Dataset):
    def __init__(self, size=500):
        self.data = []
        self.labels = []
        for _ in tqdm(range(size)):
            theta = [np.random.uniform(0, 2 * np.pi) for i in range(30)]
            noise_factor,result = extra_polation(random_circuit,theta,noise_factor=[0,1,2,3],p1=0.01,p2=0.02)
            self.data.append(np.array(noise_factor))
            self.labels.append(result)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

# 데이터셋 및 데이터 로더 생성
dataset = QuantumData()
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

  0%|          | 0/500 [00:00<?, ?it/s]

  4%|▍         | 22/500 [00:21<10:13,  1.28s/it]

In [ ]:
import torch.nn as nn

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 1)
        self.relu = nn.GELU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return torch.squeeze(self.fc3(x))

model = RegressionModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

: 

In [ ]:
from torch.utils.data import random_split, DataLoader
# 데이터셋 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 데이터 로더
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# 모델, 손실 함수, 옵티마이저 초기화
model = RegressionModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 훈련 및 검증 루프
num_epochs = 5000
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs_val, labels_val in val_loader:
            outputs = model(inputs_val)
            loss = criterion(outputs, labels_val)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch 1, Train Loss: 0.3895, Val Loss: 0.3306
Epoch 2, Train Loss: 0.3735, Val Loss: 0.3151
Epoch 3, Train Loss: 0.3668, Val Loss: 0.3029
Epoch 4, Train Loss: 0.3603, Val Loss: 0.2962
Epoch 5, Train Loss: 0.3506, Val Loss: 0.2901
Epoch 6, Train Loss: 0.3460, Val Loss: 0.2818
Epoch 7, Train Loss: 0.3269, Val Loss: 0.2742
Epoch 8, Train Loss: 0.3309, Val Loss: 0.2664
Epoch 9, Train Loss: 0.3212, Val Loss: 0.2558
Epoch 10, Train Loss: 0.3101, Val Loss: 0.2445
Epoch 11, Train Loss: 0.2921, Val Loss: 0.2301
Epoch 12, Train Loss: 0.2632, Val Loss: 0.2074
Epoch 13, Train Loss: 0.2413, Val Loss: 0.1838
Epoch 14, Train Loss: 0.2137, Val Loss: 0.1617
Epoch 15, Train Loss: 0.1844, Val Loss: 0.1358
Epoch 16, Train Loss: 0.1415, Val Loss: 0.1049
Epoch 17, Train Loss: 0.1091, Val Loss: 0.0751
Epoch 18, Train Loss: 0.0789, Val Loss: 0.0530
Epoch 19, Train Loss: 0.0554, Val Loss: 0.0399
Epoch 20, Train Loss: 0.0363, Val Loss: 0.0249
Epoch 21, Train Loss: 0.0295, Val Loss: 0.0224
Epoch 22, Train Loss: 

KeyboardInterrupt: 

: 

In [ ]:
model(inputs[5]),inputs[5],labels[5]

(tensor(-0.9140, grad_fn=<SqueezeBackward0>),
 tensor([-0.1987, -0.0110, -0.0012, -0.0002]),
 tensor(-0.9959))

: 

In [420]:
for i in range(12):
    print(model(inputs_val[i]).item(),labels_val[i],inputs_val[i][0])

0.4344109892845154 tensor(0.5300) tensor(0.0813)
-0.7787020802497864 tensor(-0.8143) tensor(-0.1826)
-0.7724839448928833 tensor(-0.7121) tensor(-0.1762)
0.33572620153427124 tensor(0.2991) tensor(0.0619)
-0.988797128200531 tensor(-0.9991) tensor(-0.2238)
0.2692647874355316 tensor(0.1643) tensor(0.0481)
-0.3944154381752014 tensor(-0.3738) tensor(-0.0942)
-0.8359125852584839 tensor(-0.7603) tensor(-0.1972)
-0.8393959999084473 tensor(-0.9875) tensor(-0.1908)
-1.0243483781814575 tensor(-0.9984) tensor(-0.2318)
-0.5831440687179565 tensor(-0.4724) tensor(-0.1604)
0.22117185592651367 tensor(0.1379) tensor(0.0388)
